O objetivo desse script é criar tabelas para analisar a quantidade de pedidos de patentes no Brasil de acordo com a localidade e a classe tecnológica da patente. Os dados utilizados são as bases de dados do INPI e o registro de municípios feito pelo IBGE. Links para encontrar esses dados estão abaixo:

Dados do INPI, ou BADEPI (Base de Dados sobre Propriedade Intelectual):
https://www.gov.br/inpi/pt-br/central-de-conteudo/estatisticas-e-estudos-economicos/estatisticas-1/base-de-dados-sobre-propriedade-intelectual-para-fins-estatisticos-2013-badepi

Dados do IBGE:
https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/23701-divisao-territorial-brasileira.html?=&t=downloads&utm_source=landing&utm_medium=explica&utm_campaign=codmun


In [1]:
# Bibliotecas usadas:
import pandas as pd

# Configuração para ver todas as colunas
pd.set_option('display.max_columns', None)

# Carregando e tratando os dados brasileiros

Os dados relevantes estão em quatro arquivos distintos: depositantes, depósitos, classificações e municipios. Esses arquivos serão carregados com as variáveis relevantes para a análise.

Ao final, as tabelas individuais serão reunidas em uma variável chamada 'dados'.
Comum às tabelas do INPI:

## Depositantes

### Carregando os dados

A tabela de depositantes fornece as informações dos depositantes para cada pedido de patente. Ela é relevante pois o primeiro depositante é usado para identificar a localização de cada patente. Portanto, iremos carregar os dados relativos à localização e selecioná-los para o Brasil e o primeiro depositante.

depositantes ('PTN_DEPOSITANTES.csv'):
- NO_PEDIDO, variável chave e identificador da patente
- NO_ORDEM, número atribuído ao depositante de acordo com a ordem no pedido
- PAIS, país de origem/sede do depositante
- UF, estado de origem/sede do depositante
- MUNICIPIO, município de origem/sede do depositante
- CD_IBGE_CIDADE, código do município associado

In [2]:
# carregando os dados de depositantes
depositantes = pd.read_csv(
    'dados/PTN_DEPOSITANTES.csv',
    sep = ';',
    encoding = 'latin1',
    low_memory = False,
    usecols = ['NO_PEDIDO', 'NO_ORDEM', 'PAIS', 'UF', 'MUNICIPIO', 'CD_IBGE_CIDADE']
)

# selecionando os dados para o Brasil
selecao = depositantes['PAIS'] == 'BR'
depositantes = depositantes[selecao]
depositantes.drop('PAIS', axis='columns', inplace=True)

# selecionando o primeiro depositante
selecao = depositantes['NO_ORDEM'] == 1
depositantes = depositantes[selecao]
depositantes.drop('NO_ORDEM', axis='columns', inplace=True)

depositantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150515 entries, 0 to 514623
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   NO_PEDIDO       150515 non-null  object 
 1   UF              149731 non-null  object 
 2   MUNICIPIO       134262 non-null  object 
 3   CD_IBGE_CIDADE  127955 non-null  float64
dtypes: float64(1), object(3)
memory usage: 5.7+ MB


### Tratando os dados

Os dados de depositantes não são os melhores possíveis. Há diversos dados faltantes e, em alguns casos, dados incoerentes, como o caso da patente depositada na cidade de Londrina (AC) (ver patente PI0605762, encontrada ao executar da linha de código: "dados[dados['NO_PEDIDO'] == 'PI0605762']"). Devido a esses problemas iremos usar somente os dados de patentes que possuem o código do IBGE, já que estes garantem uma localização única e coerente para cada patente. Perdemos um número razoável de dados devido a isso, mas há pouco a se fazer em relação a isso. Códigos da cidade serão usados posteriormente para unir os dados do INPI com os dados de local do IBGE.

Outro tratamento que podemos dar aos dados de depositantes é remover os pedidos duplicados, já que é esperado contar somente uma vez cada patente de acordo com seu local.

O NO_PEDIDO será feito de índice da tabela. A coluna MUNICIPIO será excluída, enquanto a coluna UF e CD_IBGE_CIDADE serão mantidas pois servirão de referência posterior para agregar os dados.

In [3]:
# removendo patentes sem o código de cidade
depositantes.dropna(subset=['CD_IBGE_CIDADE'], inplace=True)

# removendo patentes repetidas
depositantes.drop_duplicates(subset=['NO_PEDIDO'], inplace=True)

# removendo coluna dos municipios
depositantes.drop(columns=['MUNICIPIO'], inplace=True)

depositantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127874 entries, 0 to 514623
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   NO_PEDIDO       127874 non-null  object 
 1   UF              127874 non-null  object 
 2   CD_IBGE_CIDADE  127874 non-null  float64
dtypes: float64(1), object(2)
memory usage: 3.9+ MB


## Depósitos

### Carregando os dados

A tabela de depósitos é relevante pois fornece as datas de depósito de cada pedido de patente.

depositos ('PTN_DEPOSITOS.csv'):
- NO_PEDIDO, variável chave e identificador da patente
- DT_DEPOSITO, data de depósito da patente e inicial para determinar seu prazo de vigência

In [4]:
# carregando os dados de depositos
depositos = pd.read_csv(
    'dados/PTN_DEPOSITOS.csv',
    sep = ';',
    encoding = 'latin1',
    usecols = ['NO_PEDIDO', 'DT_DEPOSITO']
)

# convertendo datas para o formato datetime
depositos['DT_DEPOSITO'] = pd.to_datetime(depositos['DT_DEPOSITO'], errors='coerce')

depositos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611227 entries, 0 to 611226
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   NO_PEDIDO    611227 non-null  object        
 1   DT_DEPOSITO  611200 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 9.3+ MB


### Tratando os dados

Os tratamentos que penso ser possível para esses dados é eliminar os número de patentes repetidos e as datas de depósitos faltantes.

In [5]:
# removendo depositos de patentes duplicados
depositos.drop_duplicates(subset=['NO_PEDIDO'], inplace=True)

# removendo datas de depósitos faltantes
depositos.dropna(subset=['DT_DEPOSITO'], inplace=True)

depositos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 611200 entries, 5 to 611226
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   NO_PEDIDO    611200 non-null  object        
 1   DT_DEPOSITO  611200 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 14.0+ MB


## Classificações tecnológicas

### Carregando os dados

A tabela de classificações tecnológicas nos fornece os dados da classificação tecnológicas das patentes. Usaremos só as classificações IPC, desconsiderando as classificações fornecidas pela INPI. Também renomearemos as variáveis relativas ao código IPC para torná-las mais claras.

classificacoes ('PTN_CLASSIFICACOES.csv'):
- NO_PEDIDO, variável chave e identificador da patente
- NO_ORDEM_PEDIDO, ordem dos códigos IPC no momento de atribuição, renomeada para 'IPC_ORDEM'
- CD_CLASSIF, classificação IPC da patente, renomeada para 'IPC_CD'

In [6]:
# carregando os dados de classificação das patentes
classificacoes = pd.read_csv(
    'dados/PTN_CLASSIFICACOES.csv',
    sep = ';',
    usecols = ['NO_PEDIDO', 'NO_ORDEM_PEDIDO', 'CD_CLASSIF']
)

# criando novos nomes
novos_nomes = {
    'NO_ORDEM_PEDIDO': 'IPC_ORDEM',
    'CD_CLASSIF': 'IPC_CD'
}

# renomeando
classificacoes.rename(columns=novos_nomes, inplace=True)

classificacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153723 entries, 0 to 1153722
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   NO_PEDIDO  1153723 non-null  object
 1   IPC_ORDEM  1153723 non-null  int64 
 2   IPC_CD     1153723 non-null  object
dtypes: int64(1), object(2)
memory usage: 26.4+ MB


### Tratando os dados

Não foi identificada nenhum tratamento de dados possível para as classificações tecnológicas.

## Divisões do território segundo o IBGE

### Carregando os dados

Por fim temos a tabela do IBGE. Esses dados serão incorporados para facilitar a agregação por níveis diferentes de localidade, sendo a mais ampla por unidades da federação, passando pelas regiões intermediárias e imediatas, seguindo o novo recorte divulgado pelo IBGE em 2017, ou as antigas mesorregiões e microrregiões, até chegar ao município. Além do código do município, usado para unir as tabelas, serão carregados os nomes dessas subdivisões geográficas. As colunas serão renomeadas para facilitar referência posterior. De antemão é possível dizer que não há razão para um tratamento desses dados.

territorios_ibge ('RELATORIO_DTB_BRASIL_MUNICIPIO.xls'):
- Nome_UF, não será renomeado, preferível usar a variável UF dos dados do INPI
- Nome Região Geográfica Intermediária, renomeado RG_INTERMED
- Nome Região Geográfica Imediata, renomeada RG_IMEDI
- Nome_Mesorregião, renomeada MESORREGIAO
- Nome_Microrregião, renomeada MICRORREGIAO
- Código Município Completo, usada para unir as tabelas e depois removida
- Nome_Município, renomeada MUNICIPIO

In [7]:
# carregando dados dos municípios
territorios = pd.read_excel(
    'dados/RELATORIO_DTB_BRASIL_MUNICIPIO.xls',
    usecols = [
        'Nome_UF',
        'Nome Região Geográfica Intermediária',
        'Nome Região Geográfica Imediata',
        'Nome_Mesorregião',
        'Nome_Microrregião',
        'Código Município Completo',
        'Nome_Município'
    ]
)

# criando novos nomes
novos_nomes = {
    'Nome Região Geográfica Intermediária': 'RG_INTERMED',
    'Nome Região Geográfica Imediata': 'RG_IMEDI',
    'Nome_Mesorregião': 'MESORREGIAO',
    'Nome_Microrregião': 'MICRORREGIAO',
    'Nome_Município': 'MUNICIPIO'
}

# renomeando
territorios.rename(columns=novos_nomes, inplace=True)

territorios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Nome_UF                    5570 non-null   object
 1   RG_INTERMED                5570 non-null   object
 2   RG_IMEDI                   5570 non-null   object
 3   MESORREGIAO                5570 non-null   object
 4   MICRORREGIAO               5570 non-null   object
 5   Código Município Completo  5570 non-null   int64 
 6   MUNICIPIO                  5570 non-null   object
dtypes: int64(1), object(6)
memory usage: 304.7+ KB


## Unindo as tabelas

Agora que os dados foram selecionados e tratados, cabe uni-los em uma única tabela chamada 'dados' para facilitar a manipulação e execução das análises. É importante ficar atento no como essa união de tabelas será feita para não perder informações! Em cada etapa será adicionado um argumento de validação na função 'merge' do pandas. Cabe pensar se as variáveis de referência, número de pedido e código de município são únicas ou múltiplas em cada tabela.

'depositantes':
- 'NO_PEDIDO' únicos
- 'CD_IBGE_CIDADE' múltiplos

'depósitos':
- 'NO_PEDIDO' únicos

'classificacoes':
- 'NO_PEDIDO' múltiplos

'territorios':
- 'Código Município Completo' únicos

Dado que 'depositantes' já esta filtrado para o Brasil, iniciaremos unindo ela a 'territorios', garantindo que seja uma relação de muitos para um.

Depois disso, uniremos 'depositos' aos dados, em uma relação de um para um.

Os dados contidos em 'classificacoes' será o último, em uma relação de um para muitos.

Após as uniões, 'dados' conterá todas as informações relevantes.

In [8]:
# unindo depositantes e territorios em dados
dados = pd.merge(
    depositantes, territorios,
    how='left',
    left_on='CD_IBGE_CIDADE',
    right_on='Código Município Completo',
    validate='many_to_one'
)

# removendo colunas relativas aos códigos de município
dados.drop(['Código Município Completo'], axis='columns', inplace=True)

# unindo dados e depositos
dados = pd.merge(
    dados, depositos,
    how='left',
    on='NO_PEDIDO',
    validate='one_to_one'
)

# unindo dados e classificacoes
dados = pd.merge(
    dados, classificacoes,
    how='left',
    on='NO_PEDIDO',
    validate='one_to_many'
)


dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223156 entries, 0 to 223155
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   NO_PEDIDO       223156 non-null  object        
 1   UF              223156 non-null  object        
 2   CD_IBGE_CIDADE  223156 non-null  float64       
 3   Nome_UF         223156 non-null  object        
 4   RG_INTERMED     223156 non-null  object        
 5   RG_IMEDI        223156 non-null  object        
 6   MESORREGIAO     223156 non-null  object        
 7   MICRORREGIAO    223156 non-null  object        
 8   MUNICIPIO       223156 non-null  object        
 9   DT_DEPOSITO     223156 non-null  datetime64[ns]
 10  IPC_ORDEM       222138 non-null  float64       
 11  IPC_CD          222138 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(9)
memory usage: 22.1+ MB


## Tratamento final, otimização e reorganização dos dados

Observando os dados em uma única tabela, é possível observar que nem todas as patentes que estavam presentes em depósitos ou depositantes possuíam uma classificação tecnológica, algo que devemos considerar agora. Logo, removeremos essas patentes com os dados faltantes.

Além deste último tratamento dos dados, podemos otimizar alguns usos de memória. Primeiro, podemos excluir as tabelas usadas para carregar os dados antes da união. A ordem do IPC também será convertida de float para int. Outras medidas poderiam ser tomadas, mas já que os dados não estão ocupando muita memória, não complicarei demais esse processo de otimização.

Por fim, é possível reorganizar as colunas de dados para que as informações mais relevantes estejam mais visíveis. Por exemplo, é mais interessante saber a data de depósito e o código IPC dos pedidos de patentes do que a microrregião a qual ela pertence.

In [9]:
# último tratamento dos dados, removendo dados faltantes
dados.dropna(subset=['IPC_CD', 'IPC_ORDEM'], inplace=True)

# otimização de memória
del depositantes
del territorios
del depositos
del classificacoes
dados['IPC_ORDEM'] = dados['IPC_ORDEM'].astype('int')

# reorganização dos dados
dados = dados.iloc[:, [0, 9, 11, 10, 1, 8, 4, 5, 6, 7, 3, 2]]

dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 222138 entries, 0 to 223155
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   NO_PEDIDO       222138 non-null  object        
 1   DT_DEPOSITO     222138 non-null  datetime64[ns]
 2   IPC_CD          222138 non-null  object        
 3   IPC_ORDEM       222138 non-null  int32         
 4   UF              222138 non-null  object        
 5   MUNICIPIO       222138 non-null  object        
 6   RG_INTERMED     222138 non-null  object        
 7   RG_IMEDI        222138 non-null  object        
 8   MESORREGIAO     222138 non-null  object        
 9   MICRORREGIAO    222138 non-null  object        
 10  Nome_UF         222138 non-null  object        
 11  CD_IBGE_CIDADE  222138 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(9)
memory usage: 21.2+ MB


# Gerando a tabela RTA



In [10]:
# função que gera a tabela

def gerar_tabela_RTA(df, uf='', agregar_por='UF', ipc=4, inicio=1996, fim=2020):
    
    # para evitar dupla contagem, utiliza-se somente o primeiro código ipc de cada patente
    df = df[df['IPC_ORDEM'] == 1]
    
    # selecionando os dados
    if uf != '':
        df = df[df['UF'] == uf]
        
    df = df[df['DT_DEPOSITO'].dt.year >= inicio]
    df = df[df['DT_DEPOSITO'].dt.year <= fim]
    
    # criando série de referência para distribuição proporcional geral
    grupo1 = df.groupby([df['IPC_CD'].str[0:ipc]]).size()
    serie_referencia = grupo1/df.shape[0]
    
    # calculando a tabela com valores absolutos de pedidos de patentes
    grupo2 = df.groupby([agregar_por, df['IPC_CD'].str[0:ipc]], as_index=False).size()
    tabela_absolutos = grupo2.pivot(index='IPC_CD', columns=agregar_por, values='size')
    tabela_absolutos = tabela_absolutos.transform(lambda x: x.fillna(0))
    
    # tabela com valores absolutos contém a lista das regioes de acordo com a agregação...
    lista_regioes = list(tabela_absolutos.columns)
    
    # ... e serve de modelo para as tabelas com valores proporcionais e RTA!
    tabela_proporcional = tabela_absolutos
    tabela_RTA = tabela_absolutos
    
    # gerando tabela proporcional
    for regiao in lista_regioes:
        tabela_proporcional[regiao] = tabela_absolutos[regiao]/tabela_absolutos[regiao].sum()
    
    # gerando tabela RTA
    for regiao in lista_regioes:
        tabela_RTA[regiao] = (tabela_proporcional[regiao] >= serie_referencia).transform(lambda x: int(x))
    
    return tabela_RTA

In [11]:
# possíveis combinações:

# seleção por estado
ufs = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO',
       'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR',
       'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']

# seleção por anos
anos = [i for i in range(1996, 2021)]  # há dados de 1996 até 2020

# agregação por nível regional
regioes = [
    'UF',            # 'Nome_UF' também pode ser usada caso queira ver o nome completo das UFs
    'RG_INTERMED',   # Região Geográfica Intermediária
    'RG_IMEDI',      # Região Geográfica Imediata'
    'MESORREGIAO',
    'MICRORREGIAO',
    'MUNICIPIO'
]

# agregação por número de dígitos do código IPC
ipc_digitos = [1, 2, 3, 4]  # recortes do IPC acima disso são muito específicos

In [12]:
# Exemplo de tabela
tabela_RTA = gerar_tabela_RTA(
    dados,
    uf = '',
    agregar_por = 'RG_IMEDI',
    ipc = 3,
    inicio = 1996,
    fim = 2020
)

tabela_RTA

RG_IMEDI,Abaetetuba,Abaeté,Adamantina - Lucélia,Afonso Cláudio - Venda Nova do Imigrante - Santa Maria de Jetibá,Aimorés - Resplendor,Alagoinhas,Alegre,Alfenas,Almenara,Alta Floresta,Além Paraíba,Amambai,Amparo,Andradina,Angra dos Reis,Anápolis,Apucarana,Aquidauana - Anastácio,Aracaju,Aracati,Araguaína,Arapiraca,Araranguá,Araraquara,Araras,Araripina,Araxá,Araçatuba,Araçuaí,Arcoverde,Ariquemes,Assis,Avaré,Açailândia,Açu,Bacabal,Bagé,Balsas,Barbacena,Barra do Corda,Barra do Garças,Barreiras,Barreiros - Sirinhaém,Barretos,Bauru,Belo Horizonte,Belo Jardim ¿ Pesqueira,Belém,Bento Gonçalves,Birigui - Penápolis,Blumenau,Boa Vista,Bom Jesus da Lapa,Botucatu,Bragança Paulista,Brasiléia,Breves,Brumado,Brusque,Cabo Frio,Cachoeira do Sul,Cachoeiro de Itapemirim,Cacoal,Caicó,Cajazeiras,Caldas Novas-Morrinhos,Camacan,Camaquã,Campina Grande,Campinas,Campo Belo,Campo Grande,Campo Maior,Campo Mourão,Campos dos Goytacazes,Canguaretama,Canindé,Capelinha,Caracaraí,Caraguatatuba - Ubatuba - São Sebastião,Carangola,Caratinga,Carazinho,Carpina,Caruaru,Cascavel,Castanhal,Cataguases,Catalão,Catanduva,Caxambu - Baependi,Caxias do Sul,Caçador,Ceres - Rialma - Goianésia,Cerro Largo,Chapadinha,Chapecó,Charqueadas - Triunfo - São Jerônimo,Cianorte,Colatina,Colinas do Tocantins,Conceição do Coité,Concórdia,Confresa - Vila Rica,Conselheiro Lafaiete,Cornélio Procópio ¿ Bandeirantes,Corumbá,Coxim,Criciúma,Cruz Alta,Cruz das Almas,Cruzeiro,Cruzeiro do Sul,Cuiabá,Curitiba,Curitibanos,Currais Novos,Curvelo,Cáceres,Cícero Dantas,Diamantina,Diamantino,Distrito Federal,Divinópolis,Dois Vizinhos,Dores do Indaiá,Dourados,Dracena,Encantado,Erechim,Escada - Ribeirão,Espinosa,Estância,Eunápolis - Porto Seguro,Feira de Santana,Fernandópolis,Flores de Goiás,Florianópolis,Formiga,Fortaleza,Foz do Iguaçu,Franca,Francisco Beltrão,Frederico Westphalen,Frutal,Garanhuns,Goiana ¿ Timbaúba,Goiás - Itapuranga,Goiânia,Governador Nunes Freire,Governador Valadares,Guanambi,Guanhães,Guarabira,Guarapuava,Guaratinguetá,Guaxupé,Gurupi,Ibaiti,Ibirama - Presidente Getúlio,Ijuí,Ilhéus ¿ Itabuna,Imperatriz,Inhumas - Itaberaí - Anicuns,Ipatinga,Ipiaú,Iporá,Irati,Irecê,Itabaiana,Itabira,Itacoatiara,Itaituba,Itajaí,Itajubá,Itapecuru Mirim,Itaperuna,Itapetinga,Itapetininga,Itapeva,Itapipoca,Ituiutaba,Itumbiara,Ituporanga,Iturama,Ituverava,Ivaiporã,Jaciara,Jacobina,Jales,Janaúba,Januária,Jardim,Jaru,Jataí-Mineiros,Jaú,Jequié,Ji-Paraná,Joaçaba - Herval d'Oeste,Joinville,João Câmara,João Monlevade,João Pessoa,Juara,Juazeiro,Juazeiro do Norte,Juiz de Fora,Jundiaí,Juína,Lagarto,Lages,Lagoa Vermelha,Lajeado,Laranjeiras do Sul - Quedas do Iguaçu,Lavras,Limeira,Limoeiro,Linhares,Lins,Loanda,Londrina,Luziânia,Macapá,Macaé - Rio das Ostras,Maceió,Mafra,Manaus,Manhuaçu,Manicoré,Mantena,Marabá,Marau,Maravilha,Marechal Cândido Rondon,Maringá,Marília,Miracema do Tocantins,Mirassol D'oeste,Mogi Guaçu,Monte Carmelo,Monteiro,Montenegro,Montes Claros,Mossoró,Muriaé,Natal,Naviraí - Mundo Novo,Nazaré ¿ Maragogipe,Nonoai,Nova Andradina,Nova Friburgo,Nova Prata - Guaporé,Nova Venécia,Novo Hamburgo - São Leopoldo,Oliveira,Oriximiná,Ourinhos,Palmares,Palmas,Palmeira das Missões,Palmeira dos Índios,Palmeiras de Goiás,Paragominas,Paranacity - Colorado,Paranaguá,Paranavaí,Paranaíba - Chapadão do Sul - Cassilândia,Parauapebas,Parintins,Parnaíba,Pará de Minas,Passo Fundo,Passos,Pato Branco,Patos,Patos de Minas,Patrocínio,Paulo Afonso,Pedra Azul,Pedreiras,Peixoto de Azevedo - Guarantã do Norte,Pelotas,Penedo,Petrolina,Petrópolis,Picos,Pinheiro,Piracanjuba,Piracicaba,Piraju,Pirapora,Pires do Rio,Piripiri,Pitanga,Piumhi,Pombal,Ponta Grossa,Ponta Porã,Ponte Nova,Pontes e Lacerda - Comodoro,Porangatu,Porto Alegre,Porto Nacional,Porto Velho,Posse-Campos Belos,Pouso Alegre,Poços de Caldas,Presidente Epitácio-Presidente Venceslau,Presidente Prudente,Primavera do Leste,Propriá,Quirinópolis,Recife,Redenção,Redenção-Acarape,Registro,Resende,Ribeirão Preto,Rio Bonito,Rio Branco,Rio Claro,Rio Verde,Rio de Janeiro,Rio do Sul,Rondonópolis,

## Construindo índices

In [13]:
# obtendo as colunas (subdivisões regionais)
regioes = list(tabela_RTA.columns)

# obtendo as linhas (subdivisões do código IPC)
clas_tec = list(tabela_RTA.index)

### Índice de diversidade das cidades

In [14]:
# # gerando indices de diversidade para cada regiao
# divers = {}
# for regiao in regioes:
#     divers.update({regiao: tabela_RTA[regiao].sum()})

# # mostrando os dados na tela
# for regiao in sorted(divers, key=divers.get, reverse=True):
#     print(divers[regiao], regiao)

### Índice de ubiquidade de classes tecnológicas

In [15]:
# # gerando indices de ubiquidade para cada classe tecnologica
# ubiq = {}
# for ipc in clas_tec:
#     ubiq.update({ipc: int(tabela_RTA.loc[[ipc], :].sum(axis=1))})

# # mostrando os dados na tela
# for ipc in sorted(ubiq, key=ubiq.get, reverse=True):
#     print(ubiq[ipc], ' ', ipc)

## Segundo iteração

### Ubiquidade média das tecnologias de determinada região

In [16]:
# # 
# divers2 = {}
# for regiao in regioes:
#     indice2 = 0
#     for ipc in clas_tec:
#         indice2 += ubiq[ipc] * int(tabela_RTA.loc[[ipc], regiao])
#     divers2.update({regiao: indice2/divers[regiao]})
    
# # 
# for regiao in sorted(divers2, key=divers2.get, reverse=False):
#     print(regiao, divers2[regiao])

### Diversidade média das regiões com determinada tecnologia

In [17]:
# #
# ubiq2 = {}
# ipc = 'A01'

# for ipc in clas_tec:
#     indice2 = 0
#     for regiao in regioes:
#         indice2 += divers[regiao] * int(tabela_RTA.loc[[ipc], regiao])
#     ubiq2.update({ipc: indice2/ubiq[ipc]})

# # mostrando dados
# for ipc in sorted(ubiq2, key=ubiq2.get, reverse=True):
#     print(ubiq2[ipc], ' ', ipc)